# 01 Datenüberblick und SQL-Transformation

Autor: Christian Koss - ckoss@uni-bremen.de

In [1]:
# Verwendete Hard-/Software
import sys
from platform import platform
print("System-Information: \t"+platform())
print("Python-Version: \t"+sys.version.split(' ')[0])

System-Information: 	Linux-4.4.0-130-generic-x86_64-with-Ubuntu-16.04-xenial
Python-Version: 	3.5.2


In [2]:
# Verwendete externe Packages
import pandas as pd
print("Pandas-Version: \t"+pd.__version__)
import sqlite3
print("SQLite3-Version: \t"+sqlite3.sqlite_version)
import xml.etree.ElementTree as ET
print("xml.etree.ET-Version: \t"+ET.VERSION)
import re
print("RegEx-Version: \t\t"+re.__version__)
import Levenshtein #0.12.0

Pandas-Version: 	0.22.0
SQLite3-Version: 	3.11.0
xml.etree.ET-Version: 	1.3.0
RegEx-Version: 		2.2.1


In [3]:
# Datensätze
import os
data_path="data"
os.listdir(data_path)

['amazon', 'data.db', 'wikipedia']

# Amazon (Webis-CLS-10)

Original-Quelle der Daten:<br/>
[amazon.de](http://amazon.de)

Datenerhebung: <br/>
Prettenhofer, P., & Stein, B. (2010). Cross-language text classification using structural correspondence learning. In Proceedings of the 48th annual meeting of the association for computational linguistics, Association for Computational Linguistics, S. 1118-1127
<br/><br/>
[Datasets](https://www.uni-weimar.de/de/medien/professuren/medieninformatik/webis/data/webis-cls-10/)<br/>
[Publikation](https://aclanthology.info/pdf/P/P10/P10-1114.pdf)<br/>
[Read Me](https://www.uni-weimar.de/medien/webis/corpora/corpus-webis-cls-10/cls-acl10-unprocessed-README.txt)<br/>
<br/>
(Zugriff 04.04.2018)

In [74]:
data_name="amazon"
os.listdir(data_path+'/'+data_name)

['amazon.db', 'music', 'books', 'dvd']

In [75]:
# XML -> Pandas
paths = [data_path+"/"+data_name+"/"+path+"/unlabeled.review" for path in ["music","books","dvd"]]
music_path, books_path, dvd_path = paths

import xml.etree.ElementTree as ET
dbs={}
for path in paths:
    tree = ET.parse(path)
    root = tree.getroot()
    db = path.split('/')[2]
    dbs[db]=pd.DataFrame([[i]+[child.text for child in root[i]] for i in range(len(root))], columns=['ID']+[child.tag for child in root[0]])
    print(db+': \t'+str(len(root.findall("./item"))))

music: 	60392
books: 	165470
dvd: 	91516


In [76]:
dbs['dvd'][:3]

,ID,category,rating,realname,asin,url,text,title,summary,date,location,helpfulness_votes,reviewer
0,0,DVD &amp; Blu-ray,2.0,False,B002GH1M5W,http://www.amazon.de/product-reviews/B002GH1M5W/,X MEN ORIGINS : WOLVERINE präsentiert uns die ...,X-Men Origins: Wolverine (Extended Version ink...,"""...da ist eben ein nackter Mann in unsere Sch...",15. Oktober 2009,None,None,Caligula
1,1,DVD &amp; Blu-ray,5.0,False,B001G7K8QW,http://www.amazon.de/product-reviews/B001G7K8QW/,Komme gerade aus dem Kino. Großartiger Film.\n...,Willkommen bei den Sch'tis,"Nicht nur für Blödbommel und Zipfel, Häää!",2. Januar 2009,"Dortmund, Westfalen",None,Pixie
2,2,DVD &amp; Blu-ray,2.0,False,B002ACP13M,http://www.amazon.de/product-reviews/B002ACP13M/,Für mich und meine Lebensgefährtin war der Fil...,Illuminati,Greift lieber zum Buch oder zum Hörbuch!,25. Juli 2009,None,None,"Curtis Newton ""curtisnewtoncf"""


In [77]:
# Pandas -> SQL

amazon_sql = data_path+'/'+data_name+'/amazon.db'
for db, df in dbs.items():   
    conn = sqlite3.connect(amazon_sql)
    cur = conn.cursor()                                 

    wildcards = ','.join(['?'] * len(df.columns))              

    cur.execute("drop table if exists %s" % db)

    col_str = '"' + '","'.join(df.columns) + '"'
    cur.execute("create table %s (%s)" % (db, col_str))

    cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

    conn.commit()
    conn.close()

# Wikipedia

Übersicht: [https://dumps.wikimedia.org/dewiki/20180820/](https://dumps.wikimedia.org/dewiki/20180820/)

Hinweis: 2018-08-22 07:44:12 done Recombine all pages, current versions only.

Download:   [dewiki-20180820-pages-meta-current.xml.bz2](https://dumps.wikimedia.org/dewiki/20180820/dewiki-20180820-pages-meta-current.xml.bz2) 30.6 GB

In [78]:
data_name="wikipedia"

os.listdir(data_path+'/'+data_name)

['dewiki-20180820-pages-meta-current.xml']

In [79]:
#==============================================================================
# wiki XML-articles
#==============================================================================
# Data-Structure
# https://www.mediawiki.org/wiki/Manual:Namespace
# https://www.mediawiki.org/wiki/Manual:Revision_table
#==============================================================================
# <page>
#     <title></title>                       # Titel der Seite
#     <redirect title />                    # Weitergeleitete Titel
#     <ns></ns>                             # Namespace: Seitentyp (Artikel/Nutzer/Hilfe/Medien/...)
#     <id></id>                             # ID des Artikels
#     <revision>                            # Meta-Daten der Editierung
#         <id></id>                             # ID der aktuellen Version
#         <parentid></parentid>                 # ID der vorherigen Version
#         <timestamp></timestamp>               # Zeit der Editierung 
#         <contributor>                         # Meta-Daten des Erstellers
#             <username></username>                 # User-Name
#             <id></id>                             # User-ID
#             <ip></ip>                             # User-IP
#         </contributor>
#         <minor />                         # Kennzeichnet minimale Editierungen
#         <comment></comment>               # Kommentar des Erstellers
#         <model></model>                   # Model des Artikels (wikitext/JS/CSS/...)
#         <format></format>                 # Format des Artikels (Wiki/HTML/PHP/...)
#         <text>                            # Inhalt/Text des Artikels (https://www.mediawiki.org/wiki/Help:Formatting)
#         </text>
#         <sha1></sha1>                     # Verschlüsselter Hash
#     </revision>
# </page>
#==============================================================================
# Example: https://www.mediawiki.org/wiki/Help:Export#Example
#==============================================================================

In [80]:
# XML -> SQL
# 6,2 Millionen Seiten

# Parse Wikipeda
wikipedia_sql = data_path+'/'+data_name+'/wikipedia.db'

ns = 'http://www.mediawiki.org/xml/export-0.10/'
db = 'wikipages'

collector=[]
memory=100000 #100.000 = bis zu 0.6GB RAM
status=True
count=0

conn=sqlite3.connect(wikipedia_sql)
cur = conn.cursor()  
var_format=[('ID','INT'),('title','TEXT'),('text','TEXT'),('infobox','TEXT')]

cur.execute("drop table if exists %s" % db)
conn.commit()

var=[x for (x,y) in var_format]
create='CREATE TABLE IF NOT EXISTS '+db+'('+', '.join(var)+')'    
wildcards = ','.join(['?'] * len(var))  
cur.execute(create) 
conn.commit()

for event, elem in ET.iterparse('data/wikipedia/dewiki-20180820-pages-meta-current.xml'):
    if elem.tag=='{%s}page' % ns:
        ID=elem.find('{%s}revision' % ns).find('{%s}id' % ns).text
        title=elem.find('{%s}title' % ns).text
        text=elem.find('{%s}revision' % ns).find('{%s}text' % ns).text
        
        try:
            infobox=re.search('\{\{Infobox([^|<}]+)',text).group(1).split('\n')[0].strip()
        except:
            infobox="None"
            
        collector.append((ID,title,text, infobox))
        elem.clear() #VERY IMPORTANT TO FREE MEMORY
        status=True
    if len(collector)>memory:
        cur.executemany("insert into %s values(%s)" % (db, wildcards), collector)
        conn.commit()
        count+=len(collector)
        collector=[]
        status=False
        
if status==True:
    cur.executemany("insert into %s values(%s)" % (db, wildcards), collector)
    conn.commit()
    count+=len(collector)
    #collector=[]
conn.close()

print("Pages: "+str(count))

Pages: 6203084


In [16]:
wikipedia_sql='data/wikipedia/wikipedia.db'
conn = sqlite3.connect(wikipedia_sql)
cur = conn.cursor() 

cur.execute('SELECT * FROM wikipages') 
data=[]
count=0
for text in cur:
    count+=1
    if count>10:
        break;
    data.append(text)
conn.close()
pd.DataFrame(data, columns=['ID','title','text','infobox'])

,ID,title,text,infobox
0,179556416,Alan Smithee,'''Alan Smithee''' steht als [[Pseudonym]] für...,None
1,175467883,Actinium,{{Infobox Chemisches Element\n<!--- Periodensy...,Chemisches Element
2,179845113,Ang Lee,[[Datei:Ang Lee - 66eme Festival de Venise (Mo...,None
3,168173779,Anschluss (Soziologie),'''Anschluss''' ist in der [[Soziologie]] ein ...,None
4,38028819,Anschlussfähigkeit,#REDIRECT [[Anschluss (Soziologie)]],None
5,179064573,Aussagenlogik,Die '''Aussagenlogik''' ist ein Teilgebiet der...,None
6,6746470,Autopoiese,#REDIRECT [[Autopoiesis]],None
7,153932060,A.A.,#redirect [[AA]],None
8,180174590,Liste von Autoren/A,__NOTOC__\n\n{{SubTOC|Titel=Liste von Autoren|...,None
9,178067098,Liste von Autoren/H,__NOTOC__\n\n{{SubTOC|Titel=Liste von Autoren|...,None


# Wiktionary

Übersicht: [https://dumps.wikimedia.org/dewiktionary/20180901/](https://dumps.wikimedia.org/dewiktionary/20180901/)

Hinweis: 2018-09-02 19:23:57 done Articles, templates, media/file descriptions, and primary meta-pages

Download: [dewiktionary-20180901-pages-articles-multistream.xml.bz2](https://dumps.wikimedia.org/dewiktionary/20180901/dewiktionary-20180901-pages-articles-multistream.xml.bz2) 153.6 MB

In [ ]:
# XML -> SQL
# 625617 n-gram

# Parse Wiktionary
wiktionary_sql = 'data/wiktionary/wiktionary.db'
conn=sqlite3.connect(wiktionary_sql)
cur = conn.cursor()
cur.execute('SELECT title, text FROM wikipages')

bad_pages=['Hilfe:',
            'Kategorie:',
            'Modul:',
            'Reim:',
            'Verzeichnis:',
            'Vorlage:',
            'Flexion:',
            'Wiktionary:']

stemm={}
pos={}
synonym={}
related={}
gender={}
subs={}
token={}
extra={}
wortbildung={}
cat={}
lemma={}

#for title,text in cur:
#    if 'Verzeichnis:Deutsch/Wortbildungen/' in title:
#        print(title,text)
#        break;

for title,text in cur:
    if text is not None and title.split(':')[0]+':' not in bad_pages and ('{{Sprache|Deutsch}}' in text or '|Deutsch}}' in text.split('===\n')[0] or '{{Alte Schreibweise' in text.split('===\n')[0]):
        for tag in re.findall('{{[^\{\|]{4,}}}',text):
            try:
                cat[tag]+=1
            except:
                cat[tag]=1
        temp=title.split('==')[0]
        token[temp]=None
        text=text.split('{{Sprache|Deutsch}}')[-1].split('== '+temp+' ({{')[0]
        
        #Stamm
        for tag in ['{{Grundformverweis','{{Lemmaverweis','{{Alte Schreibweise']:
            try:
                stemm[temp]=re.search(tag+'[^|]+(.*?)}}', text).group(1).split('|')[1].strip()
                break;
            except:
                pass

        #Gender
        for tag in ['{{Weibliche Wortformen}}','{{Männliche Wortformen}}','{{Sächliche Wortformen}}']:
            templist=[]
            try:
                templist+=re.findall('\[\[([^[]+)\]\]',re.search(tag+'[^{]+', text).group(0))
            except:
                pass
            if len(templist)>0:
                gender[temp]=templist
                
        #POS   
        try:
            pos[temp]=re.search('{{Wortart[|]([^|]+)', text).group(1).split('}')[0].split('<!')[0].split('(')[0].strip()
        except:
            pass
        
        #Lemmas
        try:
            synonym[temp]=re.findall('\[\[([^[]+)\]\]',re.search('{{Synonyme}}[^{]+', text).group(0))
        except:
            pass
        
        try:
            related[temp]=re.findall('\[\[([^[]+)\]\]',re.search('{{Sinnverwandte Wörter}}[^{]+', text).group(0))
        except:
            pass
        try:
            subs[temp]=re.findall('\[\[([^[]+)\]\]',re.search('{{Unterbegriffe}}[^{]+', text).group(0))
        except:
            pass
        
        try:
            try:
                a=synonym[temp]
            except:
                a=[]
            try:
                b=related[temp]
            except:
                b=[]
            try:
                c=subs[temp]
            except:
                c=[]
            try:
                d=gender[temp]
            except:
                d=[]
                
            for word in a+b+c+d:
                try:
                    lemma[word]+=[temp]
                except:
                    lemma[word]=[temp]
                    token[word]=None
        except:
            pass
            
        
        #extra
        try:
            wortbildung[temp]=[(a+b,c) for a,b,c in re.findall(""":''\[\[([A-Za-zäöüßÄÖÜ]+)\]\]:''|(:)|\[\[([^[]+)\]\]""",re.sub("""\]\]e:''""","""]]:''""",re.search('{{Wortbildungen}}([^{]+)', text).group(1)).replace('\n',''))]
        except:
            pass
        
conn.close()

examples={}
for k,v in pos.items():
    try:
        if len(examples[v])<3:
            examples[v]+=[k]
    except:
        examples[v]=[k]

In [ ]:
# Versuche unbekannte Wörter (rote Links) mit aufzunehmen
unknown={}
count=0
for tok, wordlist in wortbildung.items():
    for tag, word in wordlist:
        try:
            pos[word]
        except:
            tag=tag.split('/')[0]
            word=word.split('/')[0]
            if tag==':':
                count+=1
                temp=count
            if tag!=':' and tag!='':
                temp=tag
                
            temp0=word.split(tok)    
            if len(temp0)==2 and len(temp0[0])>0 and len(temp0[1])==0 and pos[tok]=='Verb':
                pos[word]=pos[tok]
                token[word]=None
    
            else:
                if tag=='' and type(temp) is int:
                    if word[0].isupper():
                        try:
                            if pos[word]=='' or pos[word]==None:
                                pos[word]
                                token[word]=None
                        except:
                            pos[word]='Substantiv'
                            token[word]=None
                    else:
                        try:
                            unknown[temp]+=[(tok, temp,word)]
                        except:
                            unknown[temp]=[(tok, temp,word)]

                if tag=='' and type(temp) is not int:
                    try:
                        if pos[word]=='' or pos[word]==None:
                            pos[word]=temp
                            token[word]=None
                    except:
                        pos[word]=temp
                        token[word]=None
    wortbildung[tok]=[word for tag,word in wordlist if word!='']

In [ ]:
data=[]
for tok in token:
        try:
            pos0=pos[tok]
        except:
            pos0='None'
        
        try:
            stemm0=stemm[tok]
        except:
            stemm0='None'
            
        try:
            #lemmas={}
            #for l in lemma[tok]:
            #    try:
            #        lemmas.update({k:None for k in lemma[l]})
            #    except:
            #        pass
            lemma0=lemma[tok]
        except:
            lemma0=['None']
            
        if lemma0=='':
            lemma0=['None']
            
        line=[tok.split('|')[0],pos0,stemm0,'|'.join(lemma0)]
        data.append(line)
            
df=pd.DataFrame(data, columns=['token','pos','stemm','lemma'])
df[:5]

In [ ]:
db='lexikon' 
conn = sqlite3.connect(wiktionary_sql)
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()